In [1]:
from keras.datasets import cifar100
import matplotlib.pyplot as plt
import numpy as np
 
# Download dataset of CIFAR-100 (Canadian Institute for Advanced Research)
(X_train,Y_train),(X_test,Y_test) = cifar100.load_data()
 
# Check the shape of the array
print('x_train shape:', X_train.shape)
print('x_test shape:', X_test.shape)
print('y_train shape:', Y_train.shape)
print('y_test shape:', Y_test.shape)
 
# Number of data set samples 
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')
 
# Data format confirmation
print(type(X_test))
print(type(Y_test[0]))

2022-08-04 15:03:29.876199: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-08-04 15:03:30.141069: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-04 15:03:30.141097: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


x_train shape: (50000, 32, 32, 3)
x_test shape: (10000, 32, 32, 3)
y_train shape: (50000, 1)
y_test shape: (10000, 1)
50000 train samples
10000 test samples
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [27]:
from tensorflow import keras
from tensorflow.keras import layers
from keras.constraints import maxnorm
from keras.models import Sequential
from keras.utils import np_utils
import keras.models
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout

In [2]:
# Normalize the inputs from 0-255 to between 0 and 1 by dividing by 255
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = X_train / 255.0
X_test = X_test / 255.0

In [6]:
# One-hot encode outputs
y_train = np_utils.to_categorical(Y_train)
y_test = np_utils.to_categorical(Y_test)
class_num = y_test.shape[1]

Create the model

In [33]:
model = keras.Sequential()
#model.add(keras.layers.Conv2D(32, (3, 3), input_shape=X_train.shape[1:], padding='same'))
#model.add(keras.layers.Activation('relu'))

In [34]:
model.add(keras.layers.Conv2D(32, 3, input_shape=(32, 32, 3), activation='relu', padding='same'))

In [35]:
model.add(keras.layers.Dropout(0.2))

In [36]:
model.add(keras.layers.BatchNormalization())

In [37]:
model.add(keras.layers.Conv2D(64, 3, activation='relu', padding='same'))
model.add(keras.layers.MaxPooling2D(2))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.BatchNormalization())

In [38]:
model.add(keras.layers.Conv2D(64, 3, activation='relu', padding='same'))
model.add(keras.layers.MaxPooling2D(2))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.BatchNormalization())
    
model.add(keras.layers.Conv2D(128, 3, activation='relu', padding='same'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.BatchNormalization())

In [39]:
model.add(keras.layers.Flatten())
model.add(keras.layers.Dropout(0.2))

In [40]:
model.add(keras.layers.Dense(32, activation='relu'))
model.add(keras.layers.Dropout(0.3))
model.add(keras.layers.BatchNormalization())

In [41]:
model.add(keras.layers.Dense(class_num, activation='softmax'))

In [42]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', 'val_accuracy'])


In [43]:
print(model.summary())

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 32, 32, 32)        896       
                                                                 
 activation_7 (Activation)   (None, 32, 32, 32)        0         
                                                                 
 conv2d_10 (Conv2D)          (None, 32, 32, 32)        9248      
                                                                 
 dropout_2 (Dropout)         (None, 32, 32, 32)        0         
                                                                 
 batch_normalization (BatchN  (None, 32, 32, 32)       128       
 ormalization)                                                   
                                                                 
 conv2d_11 (Conv2D)          (None, 32, 32, 64)        18496     
                                                      

In [48]:
seed=21

In [50]:
np.random.seed(seed)
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=25, batch_size=64)

Epoch 1/25


ValueError: in user code:

    File "/home/daniel/anaconda3/lib/python3.9/site-packages/keras/engine/training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "/home/daniel/anaconda3/lib/python3.9/site-packages/keras/engine/training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/daniel/anaconda3/lib/python3.9/site-packages/keras/engine/training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "/home/daniel/anaconda3/lib/python3.9/site-packages/keras/engine/training.py", line 894, in train_step
        return self.compute_metrics(x, y, y_pred, sample_weight)
    File "/home/daniel/anaconda3/lib/python3.9/site-packages/keras/engine/training.py", line 987, in compute_metrics
        self.compiled_metrics.update_state(y, y_pred, sample_weight)
    File "/home/daniel/anaconda3/lib/python3.9/site-packages/keras/engine/compile_utils.py", line

In [47]:
# Model evaluation
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

ValueError: in user code:

    File "/home/daniel/anaconda3/lib/python3.9/site-packages/keras/engine/training.py", line 1557, in test_function  *
        return step_function(self, iterator)
    File "/home/daniel/anaconda3/lib/python3.9/site-packages/keras/engine/training.py", line 1546, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/daniel/anaconda3/lib/python3.9/site-packages/keras/engine/training.py", line 1535, in run_step  **
        outputs = model.test_step(data)
    File "/home/daniel/anaconda3/lib/python3.9/site-packages/keras/engine/training.py", line 1502, in test_step
        return self.compute_metrics(x, y, y_pred, sample_weight)
    File "/home/daniel/anaconda3/lib/python3.9/site-packages/keras/engine/training.py", line 987, in compute_metrics
        self.compiled_metrics.update_state(y, y_pred, sample_weight)
    File "/home/daniel/anaconda3/lib/python3.9/site-packages/keras/engine/compile_utils.py", line 4

In [44]:
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.metrics import f1_score

In [46]:
cm=confusion_matrix(model,Y_test) ## To obtain the confusion matrix

TypeError: Expected sequence or array-like, got <class 'keras.engine.sequential.Sequential'>

In [ ]:
import numpy as np

In [24]:
model.add(Conv2D(128, (3, 3), padding='same',
    input_shape=X_train.shape[1:]))
model.add(Activation('elu'))
model.add(Conv2D(128, (3, 3)))
model.add(Activation('elu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.1))

In [25]:
model.add(Conv2D(256, (3, 3), padding='same'))
model.add(Activation('elu'))
model.add(Conv2D(256, (3, 3)))
model.add(Activation('elu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(512, (3, 3), padding='same'))
model.add(Activation('elu'))
model.add(Conv2D(512, (3, 3)))
model.add(Activation('elu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

ValueError: Exception encountered when calling layer "conv2d_8" (type Conv2D).

Negative dimension size caused by subtracting 3 from 2 for '{{node conv2d_8/Conv2D}} = Conv2D[T=DT_FLOAT, data_format="NHWC", dilations=[1, 1, 1, 1], explicit_paddings=[], padding="VALID", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true](Placeholder, conv2d_8/Conv2D/ReadVariableOp)' with input shapes: [?,2,2,512], [3,3,512,512].

Call arguments received by layer "conv2d_8" (type Conv2D):
  • inputs=tf.Tensor(shape=(None, 2, 2, 512), dtype=float32)

In [16]:
model = keras.Sequential([
    keras.layers.layer1,
    keras.layers.layer2,
    keras.layers.layer3    
])

AttributeError: module 'keras.layers' has no attribute 'layer1'